In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

df = pd.read_csv("npf_train.csv")

df["event"] = df["class4"] != "nonevent"
df = df.drop(["class4"], axis = 1)
y = df["event"].values



#Remove columns that we don't want to use.
columns = df.columns.values.tolist()
remove = ["id", "date", "event"]
columns = [column for column in columns if column not in remove]

X = df[columns].values


model = MultinomialNB(min)

X, X_test, y, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_validate, y_train, y_validate = train_test_split(X, y, train_size= 0.75)

model.fit(X_train, y_train)

Hello
